In [1]:
pip install --upgrade gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 12.2 MB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 38.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 34.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Downloading pre-trained word2vec embeddings

In [2]:
import gensim.downloader
google_news_vec = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


# Question 1.1

### a)

In [16]:
most_similar_word = google_news_vec.most_similar('student')[0][0]
most_similar_score = google_news_vec.most_similar('student')[0][1]
print(f"The most similar word to the word 'student' is {most_similar_word} and the cosine similarity is {most_similar_score}")

The most similar word to the word 'student' is students and the cosine similarity is 0.7294867038726807


### b)

In [18]:
most_similar_word = google_news_vec.most_similar('Apple')[0][0]
most_similar_score = google_news_vec.most_similar('Apple')[0][1]
print(f"The most similar word to the word 'Apple' is {most_similar_word} and the cosine similarity is {most_similar_score}")

The most similar word to the word 'Apple' is Apple_AAPL and the cosine similarity is 0.7456986308097839


### c)

In [19]:
most_similar_word = google_news_vec.most_similar('apple')[0][0]
most_similar_score = google_news_vec.most_similar('apple')[0][1]
print(f"The most similar word to the word 'apple' is {most_similar_word} and the cosine similarity is {most_similar_score}")

The most similar word to the word 'apple' is apples and the cosine similarity is 0.720359742641449


# Question 1.2

### a) Training File Size: 14041, Development File Size: 3250, Test File Size: 3453

In [99]:
train_lines = open('data/eng.train','r').read().splitlines()
dev_lines = open('data/eng.testa','r').read().splitlines()
test_lines = open('data/eng.testb','r').read().splitlines()


In [100]:
# The original data files have -DOCSTART- lines used to separate documents --> Extra handling required
train_lines

['EU NNP I-NP I-ORG',
 'rejects VBZ I-VP O',
 'German JJ I-NP I-MISC',
 'call NN I-NP O',
 'to TO I-VP O',
 'boycott VB I-VP O',
 'British JJ I-NP I-MISC',
 'lamb NN I-NP O',
 '. . O O',
 '',
 'Peter NNP I-NP I-PER',
 'Blackburn NNP I-NP I-PER',
 '',
 'BRUSSELS NNP I-NP I-LOC',
 '1996-08-22 CD I-NP O',
 '',
 'The DT I-NP O',
 'European NNP I-NP I-ORG',
 'Commission NNP I-NP I-ORG',
 'said VBD I-VP O',
 'on IN I-PP O',
 'Thursday NNP I-NP O',
 'it PRP B-NP O',
 'disagreed VBD I-VP O',
 'with IN I-PP O',
 'German JJ I-NP I-MISC',
 'advice NN I-NP O',
 'to TO I-PP O',
 'consumers NNS I-NP O',
 'to TO I-VP O',
 'shun VB I-VP O',
 'British JJ I-NP I-MISC',
 'lamb NN I-NP O',
 'until IN I-SBAR O',
 'scientists NNS I-NP O',
 'determine VBP I-VP O',
 'whether IN I-SBAR O',
 'mad JJ I-NP O',
 'cow NN I-NP O',
 'disease NN I-NP O',
 'can MD I-VP O',
 'be VB I-VP O',
 'transmitted VBN I-VP O',
 'to TO I-PP O',
 'sheep NN I-NP O',
 '. . O O',
 '',
 'Germany NNP I-NP I-LOC',
 "'s POS B-NP O",
 'rep

In [101]:
test_lines

['SOCCER NN I-NP O',
 '- : O O',
 'JAPAN NNP I-NP I-LOC',
 'GET VB I-VP O',
 'LUCKY NNP I-NP O',
 'WIN NNP I-NP O',
 ', , O O',
 'CHINA NNP I-NP I-PER',
 'IN IN I-PP O',
 'SURPRISE DT I-NP O',
 'DEFEAT NN I-NP O',
 '. . O O',
 '',
 'Nadim NNP I-NP I-PER',
 'Ladki NNP I-NP I-PER',
 '',
 'AL-AIN NNP I-NP I-LOC',
 ', , O O',
 'United NNP I-NP I-LOC',
 'Arab NNP I-NP I-LOC',
 'Emirates NNPS I-NP I-LOC',
 '1996-12-06 CD I-NP O',
 '',
 'Japan NNP I-NP I-LOC',
 'began VBD I-VP O',
 'the DT I-NP O',
 'defence NN I-NP O',
 'of IN I-PP O',
 'their PRP$ I-NP O',
 'Asian JJ I-NP I-MISC',
 'Cup NNP I-NP I-MISC',
 'title NN I-NP O',
 'with IN I-PP O',
 'a DT I-NP O',
 'lucky JJ I-NP O',
 '2-1 CD I-NP O',
 'win VBP I-VP O',
 'against IN I-PP O',
 'Syria NNP I-NP I-LOC',
 'in IN I-PP O',
 'a DT I-NP O',
 'Group NNP I-NP O',
 'C NNP I-NP O',
 'championship NN I-NP O',
 'match NN I-NP O',
 'on IN I-PP O',
 'Friday NNP I-NP O',
 '. . O O',
 '',
 'But CC O O',
 'China NNP I-NP I-LOC',
 'saw VBD I-VP O',
 

In [102]:
def count_sentences(lines):
    count=0
    for idx,line in enumerate(lines):
        if line=='':
            count+=1
        # Handle both cases of -DOCSTART-
        elif line=='-DOCSTART- -X- O O' or line=='-DOCSTART- -X- -X- O':
            #If reach end of line continue
            if idx+1==len(lines):
                continue
            else:
                # If next line followed by -DOCSTART- -X- O O is a newline minus count by 1
                if lines[idx+1]=='':
                    count-=1
    return count

In [105]:
train_count = count_sentences(train_lines)
dev_count = count_sentences(dev_lines)
test_count = count_sentences(test_lines)

print(f"Training File Size: {train_count}, Development File Size: {dev_count}, Test File Size: {test_count}." )

Training File Size: 14041, Development File Size: 3250, Test File Size: 3453.


### b)

In [225]:
# Fifth sentence in the training data set has at least two named entities with more than one word

#Extracting the fifth sentence manually
sample_sentence = train_lines[47:78]
sample_sentence

['Germany NNP I-NP I-LOC',
 "'s POS B-NP O",
 'representative NN I-NP O',
 'to TO I-PP O',
 'the DT I-NP O',
 'European NNP I-NP I-ORG',
 'Union NNP I-NP I-ORG',
 "'s POS B-NP O",
 'veterinary JJ I-NP O',
 'committee NN I-NP O',
 'Werner NNP I-NP I-PER',
 'Zwingmann NNP I-NP I-PER',
 'said VBD I-VP O',
 'on IN I-PP O',
 'Wednesday NNP I-NP O',
 'consumers NNS I-NP O',
 'should MD I-VP O',
 'buy VB I-VP O',
 'sheepmeat NN I-NP O',
 'from IN I-PP O',
 'countries NNS I-NP O',
 'other JJ I-ADJP O',
 'than IN I-PP O',
 'Britain NNP I-NP I-LOC',
 'until IN I-SBAR O',
 'the DT I-NP O',
 'scientific JJ I-NP O',
 'advice NN I-NP O',
 'was VBD I-VP O',
 'clearer JJR I-ADJP O',
 '. . O O']

In [219]:
# Function to form complete named entities from the label for each words and all the named entities in this sentence
def form_named_entities(sentence):
    words = []
    labels = []
    for line in sentence:
        words.append(line.split()[0])
        labels.append(line.split()[-1])
    named_entities = []
    is_prev_added = False
    temp_word=None
    for i in range(1, len(labels)):
        if (labels[i] == labels[i-1]) and (labels[i-1] != 'O' and labels[i]!= 'O'):
            # If nothing was added to named_entities in the previous loop --> Add current word and the word before
            if not is_prev_added:
                temp_word = ' '.join(words[i-1:i+1])
                is_prev_added=True
            else:
                temp_word = ' '.join([temp_word,words[i]])
        else:
            if is_prev_added:
                named_entities.append(temp_word)
            is_prev_added=False
    
    # Handle edge case where the last few words are named entities
    if is_prev_added:
        named_entities.append(temp_word)
            
    return named_entities

In [223]:
named_entities = form_named_entities(sample_sentence)

In [224]:
named_entities

['European Union', 'Werner Zwingmann']

# Question 1.3

### a)